Proyecto 8. Megaline: Smart o Ultra.

Paso 1. importar los datasets utilizados.

Primero importaremos las librerías correspondiente. Luego de esto leemos el archivo como valores separados por comas (ambos son CSV, de acuerdo a la terminación de los archivos).

In [4]:
#Librerías
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [5]:
##Leemos el archivo
us_behave=pd.read_csv('/data/users_behavior.csv')

Paso 2. Estudiar y observar los datos que contiene y los tipos de datos.


Se imprimen las primeras filas para verificar cual es el delimitador del archivo, como están los nombres de las columnas, entre otras cosas. Se lee la información general usando el método "info()", observando la presencia de datos nulos y el tipo de datos que tenemos en los data frames.


In [8]:
#Mostramos la información del Data frame de us_behave
display(us_behave)
us_behave.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


No hay datos nulos. Las primeras cuatro columnas (calls, minutes, messages, mb_used) son datos de tipo float64. Las columnas calls y messages podrían ser transformadas a int64, por el momento no lo haremos. Para la columna is_ultra en esta se muestra el plan para el mes actual (Ultra es 1, Smart es 0). Esta columna tiene datos de tipo int64.

2.- Segmentar los datos fuente en tres conjuntos: entrenamiento (60%), validación (20%) y prueba (20%).

In [11]:
# Dividir los datos en características y objetivo
features = us_behave.drop(columns=['is_ultra'])
target = us_behave['is_ultra']

In [12]:
# Dividimos los datos en dos partes: 80% para entrenamiento+validación y 20% para prueba
features_train_val, features_test, target_train_val, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)

# Dividimos el conjunto de entrenamiento+validación en dos partes: 
#75% (60% del total original) para entrenamiento y 25% (20% del total original) para validación
features_train, features_val, target_train, target_val = train_test_split(features_train_val, target_train_val, test_size=0.25, random_state=12345)

3.- Investigaremos la calidad de diferentes modelos cambiando los hiperparámetros.

1.-Bosque aleatorio. Para este modelo determinamos best_score igual a 0 y el número de estimadores (best_est) en 0 árboles de nuestro bosque. De esta manera se itera por los árboles de nuestro rango del hiperparámetro. El modelo se configura por el número de árboles, con el estado random de 12345. Se entrena el modelo con el conjunto de entrenamiento. Se calcula la puntuación de exactitud usando el conjunto de validación. De está forma obtenemos el mejor modelo y su exactitud, con el número de estimadores.

In [48]:
best_score = 0
best_est = 0

for est in range(1, 101): 
    model_for = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_for.fit(features_train, target_train)
    score = model_for.score(features_val, target_val)
    if score > best_score:
        best_score = score
        best_est = est

print("La exactitud del mejor modelo usando el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

La exactitud del mejor modelo usando el conjunto de validación (n_estimators = 65): 0.7993779160186625


La puntuación de exactitud del mejor modelo (con 65 árboles ó estimadores) usando el conjunto de validación es de 0.7993779160186625. 


2.-Regresión logística. Se determina el modelo usando el estado random de 12345 y se selecciona el algoritmo de optimización de liblinear. Este último funciona bien con data sets pequeños, y maneja solamente clasificación binaria. Se entrena el modelo usando el conjunto de entrenamiento. Se muestran la exactitud de los modelos de regresión logistica usando los conjuntos de entrenamiento y validación. 

In [49]:
model_lr = LogisticRegression(random_state=12345, solver='liblinear')

model_lr.fit(features_train, target_train) 

score_train = model_lr.score(features_train, target_train)
score_valid = model_lr.score(features_val, target_val) 

print("Exactitud del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Exactitud del modelo de regresión logística en el conjunto de validación:", score_valid)

Exactitud del modelo de regresión logística en el conjunto de entrenamiento: 0.7028008298755186
Exactitud del modelo de regresión logística en el conjunto de validación: 0.6998444790046656


La exactitud del conjunto de entrenamiento es mayor a los valores de exxactitud en los conjuntos de validación. Esto indica un sobreajuste. La exactitud usando el conjunto de validación es menor a comparación del obtenido usando el modelo de bosque aleatorio.

4.-Comprueba la calidad del modelo usando el conjunto de prueba.


1.- Bosque aleatorio

In [52]:
model_test_for = RandomForestClassifier(random_state=12345, n_estimators=65)
model_test_for.fit(features_train, target_train)
score_test_for = model_test_for.score(features_test, target_test)

print("Puntuación de exactitud usando el conjunto de prueba, con los 65 estimadores", score_test_for)

Puntuación de exactitud usando el conjunto de prueba, con los 65 estimadores 0.7900466562986003


Nuestro modelo muestra indicios de sobreajsute. La exactitud obtenida usando el conjunto de prueba es de 0.7900466562986003. Esto es menor a comparación de la puntuación obtenida usando el conjunto de validación: 0.7993779160186625. Esto es comprensible, debido a que estamos usando una quinta parte de la cantidad de datos iniciales, con la cual hicimos dicha prueba.

2.- Regresión logística

In [59]:
score_test=model_lr.score(features_test, target_test)
print("Exactitud del modelo de regresión logística en el conjunto de prueba:", score_test)

Exactitud del modelo de regresión logística en el conjunto de prueba: 0.6998444790046656


De la misma forma, el modelo que usa la regresión logística parece tener un ligero sobreajuste. Comparando ambos modelos, observamos que el mejor modelo para trabajar con nuestros datos es el de el bosque aleatorio con 65 estimadores (árboles).